In [8]:
# STAGE ROUTER CONFIG
from ontology2 import ontology

ontology['function']['inference'].append({
    'name': 'stage_router',
    'description': 'This task determines the next stage in a workflow based on the current context and predefined stages.',
    'input': {
        'type': 'object',
        'properties': {
            'current_context': {
                'type': 'string',
                'description': 'Current context of the workflow, including any necessary data to decide the next stage.'
            },
            'workflow_definition': {
                'type': 'object',
                'description': 'Definition of the workflow including all stages and their configurations.'
                },
            'stage_choices': {
                'type': 'array',
                'items': {'type': 'object'},
                'description': 'List of stage names to choose from based on the current stage and its routing logic.'
            }
        },
        'required': ['current_context', 'workflow_definition', 'stage_choices']
    },
    'output': {
        'type': 'object',
        'properties': {
            'next_stage': {
                'type': 'string',
                'description': 'Name of the best stage to transition to next based on current context.',
                'enum': []  # This will be dynamically populated with the stage choices.
            }
        },
        'required': ['next_stage']
    },
    'dynamic_structured_outputs': {
        'referential': [],
        'input': [
            {
                'output_enum_path': 'properties.next_stage',
                'ontology_path': 'properties.stage_choices',
                'fetch_attributes': ['name', 'description'],
                'filter_attributes': {
                    'stage_choices': []  # This will be dynamically populated with the available stage choices.
                }
            }
        ]
    },
    'prompt': {
        'system': {
            'value': 'You are an AI assistant that helps users determine the next stage in a workflow based on the current context and predefined stages.'
        },
        'context': {
            'value': 'Analyze the current context and the workflow definition to determine the next stage in the workflow. Consider the criteria defined for each stage and ensure that the next stage meets the conditions specified.\nCurrent Context: {current_context}\nWorkflow Definition: {workflow_definition}',
            'attributes': ['current_context', 'workflow_definition']
        },
        'cognition': {
            'value': 'Analyze the current context to determine which stage in the workflow should be selected next. Evaluate the criteria for each stage based on the current context and select the appropriate stage from the stage choices provided. Each of the stages of a workflow are completed linearly from the first index of the stages array to the last, but through your recommended stage you can choose a different next step to start from, which will then continue to process linearly. Think step by step about the order of completion given the current context of the workflow, explain the pros and cons of choosing specific stages, and be critical about any dependencies which may exist in the workflow. Then, recommend a specific stage which should be chosen to go to next.',
        },
        'instructions': {
            'value': 'Using the provided current context, workflow definition, and stage choices, determine the next stage in the workflow. Ensure that the selected stage meets the criteria defined and is appropriate based on the current context.',
        }
    }
})


In [18]:
print(ontology['function']['inference'])

[{'name': 'security_finding_investigation', 'description': {'description': 'This task involves investigating security findings. The task aims to recommend context-specific actions for security professionals in different contexts.'}, 'input': {'type': 'object', 'properties': {'original_security_findings': {'type': 'string', 'description': 'The raw security findings data.'}, 'original_context': {'type': 'string', 'description': 'Additional, optional context from existing playbooks and historical data.'}}, 'required': ['original_security_findings', 'original_context']}, 'output': {'type': 'object', 'properties': {'actions_context': {'type': 'array', 'description': 'A list of any recommended actions from the list which is recommended for producing a high-quality investigation based on context so far.', 'items': {'type': 'object', 'properties': {'action': {'type': 'string', 'description': 'The recommended action to take.', 'enum': []}, 'context': {'type': 'string', 'description': 'Additiona

In [19]:
ontology['workflows'][WORKFLOW_NAME]

{'input': <function input(prompt='', /)>,
 'stages': [{'name': 'security_finding_investigation', 'stage_router': {}},
  {'name': 'security_finding_recursive_investigation',
   'stage_router': {'loopback_condition': True,
    'max_runs': 3,
    'loopback_stages': [1]}},
  {'name': 'security_finding_disposition', 'stage_router': {}}],
 'output': {'template': 'For this security finding, this was the concluded disposition: {disposition}\nThis is the message to the user: {message}\n\nThese are the recommended next steps: \n{recommended_action}\nThis is the final incident report: {report}',
  'attributes': ['disposition', 'message', 'recommended_action', 'report']}}

In [34]:
WORKFLOW_NAME = 'security_finding_investigation'

# IMAGINARY THREAD
thread_state = {
        "thread_id": '0',
        "original_thread_input": {},
        "previous_stage_index": None,
        "number_of_completions": 0,
        "mode": 'precision',
        "persona_object": {},
        "next_stage_index": 0,
        "workflow_name": WORKFLOW_NAME,
        "thread_history": [
            {
            "stage_name": 'stage name',
            "stage_input": {},
            "persona_object": {},
            "mode": 'precision',
            "is_exit": False,
            "stage_output": {}
        }
        ],
        "actions_context": [],
        "is_exit": False,
        "exit_output": None,
    }

DEFAULT_MAX_RUNS = 5

# ITERATE THROUGH STAGES
number_of_stages = len(ontology['workflows'][WORKFLOW_NAME]['stages'])
for stage_index in range(number_of_stages):
    # SET STAGE
    stage = ontology['workflows'][WORKFLOW_NAME]['stages'][stage_index]

    # GET STAGE_ROUTER CONFIGS
    loopback_condition = stage['stage_router'].get('loopback_condition',False)
    max_runs = stage['stage_router'].get('max_runs', DEFAULT_MAX_RUNS)
    loopback_stages = stage['stage_router'].get('loopback_stages', [])

    # ADD NEXT STAGE, IF EXISTS
    if stage_index != number_of_stages and stage_index not in loopback_stages:
        loopback_stages.append(stage_index)

    # VERIFY THREAD HAS NOT HAD MORE THAN MAX_RUNS OF THIS STAGE
    # COUNT STAGE RUNS
    stage_runs = 0
    for state in thread_state["thread_history"]:
        if stage['name'] == state['stage_name']:
            stage_runs += 1

    # CHECK TO SEE IF WE SHOULD RUN STAGE ROUTER
    if loopback_condition and stage_runs <= max_runs:
        
        # CREATE STAGE_CHOICES INPUT VARIABLE
        stage_choices = []

        # FROM ALL ELLIGIBLE LOOPBACK STAGES
        for loopback_stage_index in loopback_stages:

            # GET THE INFERENCE FUNCTION CONFIG
            for inference_function in ontology['function']['inference']:
                if inference_function['name'] == ontology['workflows'][WORKFLOW_NAME]['stages'][loopback_stage_index]['name']:

                    # ADD TO STAGE_CHOICES NAME/DESCRIPTION VALUES
                    stage_choices.append({'name':inference_function['name'], 'description': inference_function['description']})
        # print("Stage choices\n", stage_choices)

        # CREATE WORKFLOW_DEFINITION INPUT VARIABLE
        workflow_definition = {
            'name': WORKFLOW_NAME,
            'description': ontology['workflows'][WORKFLOW_NAME].get('description',''),
            'input': ontology['workflows'][WORKFLOW_NAME].get('input', ''),
            'stages': [],
            'output': ontology['workflows'][WORKFLOW_NAME].get('output','')
        }

        # GET INFERENCE FUNCTION CONFIG FOR EACH STAGE
        for config_stage in ontology['workflows'][WORKFLOW_NAME]['stages']:
            for inference_function in ontology['function']['inference']:
                if inference_function['name'] == config_stage['name']:
                    workflow_definition['stages'].append({
                        'name': inference_function['name'],
                        'input': inference_function['input'],
                        'output': inference_function['output'],
                        'description': inference_function['description']
                        })
                    # print("Workflow Definition\n",workflow_definition)

        # CREATE CURRENT_CONTEXT INPUT VARIABLE
        current_context = ''

        # READ THROUGH ALL OF THREAD, PULLING NAME/INPUT/OUTPUT
        for state_index in range(len(thread_state["thread_history"])):
            state = thread_state["thread_history"][state_index]
            historical_stage_name = state['stage_name']
            historical_stage_input = state["stage_input"]
            historical_stage_output = state['stage_output']
            current_context += f'Step {str(state_index)}:\nStage executed: {historical_stage_name}\nInput to the stage: {historical_stage_input}\nOutput from the stage: {historical_stage_output}\n\n'

        # print("Current Context\n",current_context)
        
        

        



Stage choices
 [{'name': 'security_finding_recursive_investigation', 'description': {'description': 'This task involves recursive investigation of security findings by examining new alert IDs correlated with the original entity. It aims to provide context-specific actions for security analysts by iterating through alerts and integrating additional context.', 'instructions': 'The task will take security findings, alert IDs, and additional context, and generate a set of recommended actions for each alert. It will determine if the investigation should continue or proceed to the next stage.'}}]
Workflow Definition
 {'name': 'security_finding_investigation', 'description': '', 'input': <built-in function input>, 'stages': [{'name': 'security_finding_investigation', 'input': {'type': 'object', 'properties': {'original_security_findings': {'type': 'string', 'description': 'The raw security findings data.'}, 'original_context': {'type': 'string', 'description': 'Additional, optional context fr

In [32]:
print(workflow_definition)

{'name': 'security_finding_investigation', 'description': '', 'input': <built-in function input>, 'stages': [{'name': 'security_finding_investigation', 'input': {'type': 'object', 'properties': {'original_security_findings': {'type': 'string', 'description': 'The raw security findings data.'}, 'original_context': {'type': 'string', 'description': 'Additional, optional context from existing playbooks and historical data.'}}, 'required': ['original_security_findings', 'original_context']}, 'output': {'type': 'object', 'properties': {'actions_context': {'type': 'array', 'description': 'A list of any recommended actions from the list which is recommended for producing a high-quality investigation based on context so far.', 'items': {'type': 'object', 'properties': {'action': {'type': 'string', 'description': 'The recommended action to take.', 'enum': []}, 'context': {'type': 'string', 'description': 'Additional context for the recommended action.'}}, 'required': ['action', 'context']}}}, '